In [7]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [9]:
import matplotlib.pyplot as plt
import seaborn as sns

In [38]:
df = pd.read_csv("Daily+temperature+data.csv")

In [39]:
df["Date"] = df["Date"].apply(lambda x: x.replace("/", "-"))

In [42]:
df["Date"] = pd.to_datetime(df["Date"], format="%m-%d-%Y")

In [45]:
df.describe()

,Date,Temperature(Min)
count,730,730.000000
mean,2017-12-31 12:00:00,11.465753
min,2017-01-01 00:00:00,0.500000
25%,2017-07-02 06:00:00,8.300000
50%,2017-12-31 12:00:00,11.300000
75%,2018-07-01 18:00:00,14.400000
max,2018-12-31 00:00:00,22.100000
std,NaN,4.106611


In [46]:
train_len = 511
train = df[:train_len]
test = df[train_len:]

In [48]:
train.shape, test.shape

((511, 2), (219, 2))

In [51]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

In [58]:
model = SimpleExpSmoothing(train["Temperature(Min)"])
model_fit = model.fit(smoothing_level=0.2, optimized=False)
test["ExpSmothing"] = model_fit.forecast(219)

In [60]:
test

,Date,Temperature(Min),ExpSmothing
511,2018-05-27,8.2,9.1997
512,2018-05-28,6.1,9.1997
513,2018-05-29,4.5,9.1997
514,2018-05-30,6.1,9.1997
515,2018-05-31,9.8,9.1997
...,...,...,...
725,2018-12-27,14.0,9.1997
726,2018-12-28,13.6,9.1997
727,2018-12-29,13.5,9.1997
728,2018-12-30,15.7,9.1997


In [62]:
from sklearn.metrics import mean_squared_error

In [63]:
np.sqrt(mean_squared_error(test["Temperature(Min)"], test["ExpSmothing"]))

3.5560737695562894

In [64]:
np.mean(abs(test["Temperature(Min)"] - test["ExpSmothing"])/test["Temperature(Min)"]*100)

30.23512912723859

In [65]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [84]:
model = ExponentialSmoothing(train["Temperature(Min)"], trend='add')
model_fit = model.fit(smoothing_level=0.5, smoothing_slope=0.01, optimized=False)
test["ExpSmothing_2"] = model_fit.forecast(219)

In [85]:
np.sqrt(mean_squared_error(test["Temperature(Min)"], test["ExpSmothing_2"]))

5.946775417494971

In [86]:
np.mean(abs(test["Temperature(Min)"] - test["ExpSmothing_2"])/test["Temperature(Min)"]*100)

41.94150134692109

In [87]:
model = ExponentialSmoothing(train["Temperature(Min)"], seasonal_periods=30, trend='add')
model_fit = model.fit(optimized=True)
test["ExpSmothing_3"] = model_fit.forecast(219)

In [88]:
np.sqrt(mean_squared_error(test["Temperature(Min)"], test["ExpSmothing_3"]))

3.9396539693099477

In [89]:
np.mean(abs(test["Temperature(Min)"] - test["ExpSmothing_3"])/test["Temperature(Min)"]*100)

34.794672290511144